# 构建评估体系

In [ ]:
"""
Text2SQL 评估系统

评估指标：
1. 精确匹配（Exact Match）：生成的SQL与标准答案完全一致
2. 执行准确率（Execution Accuracy）：生成的SQL执行结果与标准SQL结果一致
3. 有效性（Valid SQL）：生成的SQL语法正确，可以执行
4. 语义相似度：使用 LLM 评估SQL语义是否相似

数据集：sakila 数据库问答对
"""

import json
import sqlite3
import os
from openai import OpenAI
from typing import Dict, List, Tuple
import re
from difflib import SequenceMatcher
import time


class Text2SQLEvaluator:
    """Text2SQL 评估器"""
    
    def __init__(self, db_path: str, api_key: str):
        """
        初始化评估器
        
        参数:
            db_path: 数据库文件路径
            api_key: DeepSeek API密钥
        """
        self.db_path = db_path
        self.client = OpenAI(
            base_url="https://api.deepseek.com",
            api_key=api_key
        )
        
        # 获取数据库schema
        self.schema_description = self._get_schema_description()
        
        # 评估结果
        self.results = []
        
    def _get_schema_description(self) -> str:
        """获取数据库schema描述"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # 获取所有表名
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        schema_parts = ["数据库包含以下表：\n"]
        
        for (table_name,) in tables:
            if table_name.startswith('sqlite_'):
                continue
                
            schema_parts.append(f"\n{table_name}表：")
            
            # 获取表结构
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()
            
            for col in columns:
                col_id, col_name, col_type, not_null, default_val, is_pk = col
                pk_mark = " (主键)" if is_pk else ""
                schema_parts.append(f"  - {col_name} ({col_type}){pk_mark}")
        
        conn.close()
        return "\n".join(schema_parts)
    
    def _normalize_sql(self, sql: str) -> str:
        """
        标准化SQL语句，用于比较
        
        - 转换为小写
        - 移除多余空格
        - 移除末尾分号
        - 移除Markdown标记
        """
        sql = sql.strip()
        sql = sql.replace('```sql', '').replace('```', '').strip()
        sql = re.sub(r'\s+', ' ', sql)
        sql = sql.rstrip(';')
        sql = sql.lower()
        return sql
    
    def generate_sql(self, question: str) -> str:
        """
        使用LLM生成SQL语句
        
        参数:
            question: 自然语言问题
            
        返回:
            生成的SQL语句
        """
        prompt = f"""
以下是数据库的结构描述：
{self.schema_description}

用户的自然语言问题如下：
"{question}"

请注意：
1. 只返回SQL查询语句，不要包含任何其他解释、注释或格式标记（如```sql）
2. SQL语句要符合SQLite语法
3. 对于INSERT/UPDATE操作，请使用适当的默认值
"""
        
        try:
            response = self.client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": "你是一个SQL专家。请只返回SQL查询语句，不要包含任何Markdown格式或其他说明。"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0
            )
            
            sql = response.choices[0].message.content.strip()
            return self._normalize_sql(sql)
            
        except Exception as e:
            print(f"生成SQL时出错: {e}")
            return ""
    
    def execute_sql(self, sql: str) -> Tuple[bool, any]:
        """
        执行SQL语句
        
        参数:
            sql: SQL语句
            
        返回:
            (是否成功, 结果/错误信息)
        """
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        try:
            cursor.execute(sql)
            
            # 对于非SELECT查询，提交更改
            if not sql.strip().lower().startswith('select'):
                conn.commit()
                result = cursor.rowcount
            else:
                result = cursor.fetchall()
            
            conn.close()
            return True, result
            
        except Exception as e:
            conn.close()
            return False, str(e)
    
    def calculate_similarity(self, str1: str, str2: str) -> float:
        """
        计算两个字符串的相似度
        
        返回: 0-1之间的相似度分数
        """
        return SequenceMatcher(None, str1, str2).ratio()
    
    def extract_sql_components(self, sql: str) -> Dict:
        """
        提取 SQL 语句的各个组成部分
        
        返回: 包含表名、列名、关键词等的字典
        """
        sql = sql.lower()
        
        # 提取表名（FROM 和 JOIN 后面的）
        tables = set()
        table_pattern = r'(?:from|join)\s+(\w+)'
        tables.update(re.findall(table_pattern, sql))
        
        # 提取列名（SELECT 后面的，WHERE 中的等）
        columns = set()
        # SELECT 子句中的列
        select_match = re.search(r'select\s+(.*?)\s+from', sql)
        if select_match:
            select_clause = select_match.group(1)
            # 分割并清理列名
            for col in select_clause.split(','):
                col = col.strip()
                # 移除聚合函数和别名
                col = re.sub(r'\w+\(|\)|\s+as\s+\w+', '', col).strip()
                if col and col != '*':
                    columns.add(col)
        
        # WHERE 子句中的列
        where_columns = re.findall(r'\b(\w+)\s*(?:=|>|<|>=|<=|!=|like|in)', sql)
        columns.update(where_columns)
        
        # 提取 SQL 关键词
        sql_keywords = ['select', 'from', 'where', 'join', 'inner', 'left', 'right', 
                       'order by', 'group by', 'having', 'limit', 'insert', 'update', 
                       'delete', 'values', 'set', 'and', 'or', 'not', 'in', 'like', 'between']
        keywords = set()
        for keyword in sql_keywords:
            if keyword in sql:
                keywords.add(keyword)
        
        # 提取所有 tokens
        tokens = set(re.findall(r'\b\w+\b', sql))
        
        return {
            'tables': tables,
            'columns': columns,
            'keywords': keywords,
            'tokens': tokens
        }
    
    def calculate_recall_metrics(self, generated_sql: str, ground_truth_sql: str) -> Dict:
        """
        计算召回率相关指标
        
        召回率 = 标准答案中的元素在生成结果中出现的比例
        
        返回: 包含各种召回率指标的字典
        """
        gen_components = self.extract_sql_components(generated_sql)
        truth_components = self.extract_sql_components(ground_truth_sql)
        
        def safe_recall(generated_set, truth_set):
            """安全计算召回率，避免除零错误"""
            if len(truth_set) == 0:
                return 1.0  # 如果标准答案中没有该类元素，返回1
            intersection = generated_set & truth_set
            return len(intersection) / len(truth_set)
        
        def safe_precision(generated_set, truth_set):
            """安全计算精确率"""
            if len(generated_set) == 0:
                return 0.0
            intersection = generated_set & truth_set
            return len(intersection) / len(generated_set)
        
        def safe_f1(precision, recall):
            """计算F1分数"""
            if precision + recall == 0:
                return 0.0
            return 2 * (precision * recall) / (precision + recall)
        
        # 计算各个组件的召回率和精确率
        table_recall = safe_recall(gen_components['tables'], truth_components['tables'])
        table_precision = safe_precision(gen_components['tables'], truth_components['tables'])
        table_f1 = safe_f1(table_precision, table_recall)
        
        column_recall = safe_recall(gen_components['columns'], truth_components['columns'])
        column_precision = safe_precision(gen_components['columns'], truth_components['columns'])
        column_f1 = safe_f1(column_precision, column_recall)
        
        keyword_recall = safe_recall(gen_components['keywords'], truth_components['keywords'])
        keyword_precision = safe_precision(gen_components['keywords'], truth_components['keywords'])
        keyword_f1 = safe_f1(keyword_precision, keyword_recall)
        
        token_recall = safe_recall(gen_components['tokens'], truth_components['tokens'])
        token_precision = safe_precision(gen_components['tokens'], truth_components['tokens'])
        token_f1 = safe_f1(token_precision, token_recall)
        
        # 计算综合召回率（各个组件的加权平均）
        overall_recall = (table_recall * 0.3 + column_recall * 0.3 + 
                         keyword_recall * 0.2 + token_recall * 0.2)
        overall_precision = (table_precision * 0.3 + column_precision * 0.3 + 
                            keyword_precision * 0.2 + token_precision * 0.2)
        overall_f1 = safe_f1(overall_precision, overall_recall)
        
        return {
            'table_recall': table_recall,
            'table_precision': table_precision,
            'table_f1': table_f1,
            'column_recall': column_recall,
            'column_precision': column_precision,
            'column_f1': column_f1,
            'keyword_recall': keyword_recall,
            'keyword_precision': keyword_precision,
            'keyword_f1': keyword_f1,
            'token_recall': token_recall,
            'token_precision': token_precision,
            'token_f1': token_f1,
            'overall_recall': overall_recall,
            'overall_precision': overall_precision,
            'overall_f1': overall_f1,
            'generated_components': {
                'tables': list(gen_components['tables']),
                'columns': list(gen_components['columns']),
                'keywords': list(gen_components['keywords'])
            },
            'truth_components': {
                'tables': list(truth_components['tables']),
                'columns': list(truth_components['columns']),
                'keywords': list(truth_components['keywords'])
            }
        }
    
    def semantic_similarity(self, sql1: str, sql2: str, question: str) -> Tuple[bool, str]:
        """
        使用LLM评估两个SQL语句的语义相似度
        
        返回: (是否语义相似, 解释)
        """
        prompt = f"""
请评估以下两个SQL语句是否语义相同或等价。

问题: {question}

SQL 1: {sql1}
SQL 2: {sql2}

请回答：
1. 这两个SQL语句是否会产生相同的结果？
2. 如果不同，主要差异是什么？

请以JSON格式回答：{{"equivalent": true/false, "explanation": "解释"}}
"""
        
        try:
            response = self.client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": "你是SQL专家。请评估SQL语句的语义等价性。"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0
            )
            
            content = response.choices[0].message.content.strip()
            
            # 尝试解析JSON响应
            try:
                import json
                result = json.loads(content)
                return result.get("equivalent", False), result.get("explanation", "")
            except:
                # 如果无法解析JSON，使用简单的关键词匹配
                is_similar = "true" in content.lower() or "等价" in content or "相同" in content
                return is_similar, content
                
        except Exception as e:
            return False, f"评估出错: {e}"
    
    def evaluate_single(self, question: str, ground_truth_sql: str) -> Dict:
        """
        评估单个问答对
        
        返回: 评估结果字典
        """
        print(f"\n{'='*80}")
        print(f"问题: {question}")
        print(f"标准SQL: {ground_truth_sql}")
        
        # 生成SQL
        generated_sql = self.generate_sql(question)
        print(f"生成SQL: {generated_sql}")
        
        # 标准化ground truth SQL
        norm_ground_truth = self._normalize_sql(ground_truth_sql)
        
        # 1. 精确匹配
        exact_match = (generated_sql == norm_ground_truth)
        
        # 2. 字符串相似度
        string_similarity = self.calculate_similarity(generated_sql, norm_ground_truth)
        
        # 3. 召回率指标
        recall_metrics = self.calculate_recall_metrics(generated_sql, norm_ground_truth)
        
        # 4. SQL有效性
        success_gen, result_gen = self.execute_sql(generated_sql)
        success_truth, result_truth = self.execute_sql(norm_ground_truth)
        
        # 5. 执行结果匹配
        execution_match = False
        if success_gen and success_truth:
            execution_match = (result_gen == result_truth)
        
        # 6. 语义相似度（只对不精确匹配的情况评估）
        semantic_match = False
        semantic_explanation = ""
        if not exact_match and success_gen:
            semantic_match, semantic_explanation = self.semantic_similarity(
                generated_sql, norm_ground_truth, question
            )
        
        result = {
            "question": question,
            "ground_truth_sql": norm_ground_truth,
            "generated_sql": generated_sql,
            "exact_match": exact_match,
            "string_similarity": string_similarity,
            "valid_sql": success_gen,
            "execution_match": execution_match,
            "semantic_match": semantic_match,
            "semantic_explanation": semantic_explanation,
            "error": None if success_gen else result_gen,
            # 召回率相关指标
            "recall_metrics": recall_metrics
        }
        
        # 打印评估结果
        print(f"\n评估结果:")
        print(f"  ✓ 精确匹配: {'是' if exact_match else '否'}")
        print(f"  ✓ 字符串相似度: {string_similarity:.2%}")
        print(f"  ✓ SQL有效性: {'是' if success_gen else '否'}")
        print(f"  ✓ 执行结果匹配: {'是' if execution_match else '否'}")
        
        # 打印召回率指标
        print(f"\n  📊 召回率指标:")
        print(f"    - 表名召回率: {recall_metrics['table_recall']:.2%} (F1: {recall_metrics['table_f1']:.2%})")
        print(f"    - 列名召回率: {recall_metrics['column_recall']:.2%} (F1: {recall_metrics['column_f1']:.2%})")
        print(f"    - 关键词召回率: {recall_metrics['keyword_recall']:.2%} (F1: {recall_metrics['keyword_f1']:.2%})")
        print(f"    - Token召回率: {recall_metrics['token_recall']:.2%} (F1: {recall_metrics['token_f1']:.2%})")
        print(f"    - 综合召回率: {recall_metrics['overall_recall']:.2%} (F1: {recall_metrics['overall_f1']:.2%})")
        
        if not exact_match and success_gen:
            print(f"\n  ✓ 语义匹配: {'是' if semantic_match else '否'}")
            if semantic_explanation:
                print(f"    说明: {semantic_explanation[:100]}...")
        
        return result
    
    def evaluate_dataset(self, dataset: List[Dict], max_samples: int = None) -> Dict:
        """
        评估整个数据集
        
        参数:
            dataset: 问答对列表
            max_samples: 最大评估样本数（None表示全部）
            
        返回: 总体评估指标
        """
        if max_samples:
            dataset = dataset[:max_samples]
        
        print(f"\n{'='*80}")
        print(f"开始评估 {len(dataset)} 个样本")
        print(f"{'='*80}")
        
        self.results = []
        
        for i, item in enumerate(dataset, 1):
            print(f"\n[{i}/{len(dataset)}]", end=" ")
            
            result = self.evaluate_single(
                item["question"],
                item["sql"]
            )
            self.results.append(result)
            
            # 避免API限流
            time.sleep(0.5)
        
        # 计算总体指标
        metrics = self._calculate_metrics()
        self._print_summary(metrics)
        
        return metrics
    
    def _calculate_metrics(self) -> Dict:
        """计算总体评估指标"""
        total = len(self.results)
        
        if total == 0:
            return {}
        
        exact_match_count = sum(1 for r in self.results if r["exact_match"])
        valid_sql_count = sum(1 for r in self.results if r["valid_sql"])
        execution_match_count = sum(1 for r in self.results if r["execution_match"])
        semantic_match_count = sum(1 for r in self.results if r["semantic_match"] or r["exact_match"])
        
        avg_similarity = sum(r["string_similarity"] for r in self.results) / total
        
        # 计算召回率相关指标的平均值
        avg_table_recall = sum(r["recall_metrics"]["table_recall"] for r in self.results) / total
        avg_table_precision = sum(r["recall_metrics"]["table_precision"] for r in self.results) / total
        avg_table_f1 = sum(r["recall_metrics"]["table_f1"] for r in self.results) / total
        
        avg_column_recall = sum(r["recall_metrics"]["column_recall"] for r in self.results) / total
        avg_column_precision = sum(r["recall_metrics"]["column_precision"] for r in self.results) / total
        avg_column_f1 = sum(r["recall_metrics"]["column_f1"] for r in self.results) / total
        
        avg_keyword_recall = sum(r["recall_metrics"]["keyword_recall"] for r in self.results) / total
        avg_keyword_precision = sum(r["recall_metrics"]["keyword_precision"] for r in self.results) / total
        avg_keyword_f1 = sum(r["recall_metrics"]["keyword_f1"] for r in self.results) / total
        
        avg_token_recall = sum(r["recall_metrics"]["token_recall"] for r in self.results) / total
        avg_token_precision = sum(r["recall_metrics"]["token_precision"] for r in self.results) / total
        avg_token_f1 = sum(r["recall_metrics"]["token_f1"] for r in self.results) / total
        
        avg_overall_recall = sum(r["recall_metrics"]["overall_recall"] for r in self.results) / total
        avg_overall_precision = sum(r["recall_metrics"]["overall_precision"] for r in self.results) / total
        avg_overall_f1 = sum(r["recall_metrics"]["overall_f1"] for r in self.results) / total
        
        return {
            "total_samples": total,
            "exact_match_rate": exact_match_count / total,
            "valid_sql_rate": valid_sql_count / total,
            "execution_accuracy": execution_match_count / total,
            "semantic_match_rate": semantic_match_count / total,
            "average_string_similarity": avg_similarity,
            "exact_match_count": exact_match_count,
            "valid_sql_count": valid_sql_count,
            "execution_match_count": execution_match_count,
            "semantic_match_count": semantic_match_count,
            # 召回率相关指标
            "avg_table_recall": avg_table_recall,
            "avg_table_precision": avg_table_precision,
            "avg_table_f1": avg_table_f1,
            "avg_column_recall": avg_column_recall,
            "avg_column_precision": avg_column_precision,
            "avg_column_f1": avg_column_f1,
            "avg_keyword_recall": avg_keyword_recall,
            "avg_keyword_precision": avg_keyword_precision,
            "avg_keyword_f1": avg_keyword_f1,
            "avg_token_recall": avg_token_recall,
            "avg_token_precision": avg_token_precision,
            "avg_token_f1": avg_token_f1,
            "avg_overall_recall": avg_overall_recall,
            "avg_overall_precision": avg_overall_precision,
            "avg_overall_f1": avg_overall_f1
        }
    
    def _print_summary(self, metrics: Dict):
        """打印评估总结"""
        print(f"\n{'='*80}")
        print("评估总结")
        print(f"{'='*80}")
        
        print(f"\n📊 总体指标:")
        print(f"  样本总数: {metrics['total_samples']}")
        print(f"\n  精确匹配率: {metrics['exact_match_rate']:.2%} ({metrics['exact_match_count']}/{metrics['total_samples']})")
        print(f"  SQL有效率: {metrics['valid_sql_rate']:.2%} ({metrics['valid_sql_count']}/{metrics['total_samples']})")
        print(f"  执行准确率: {metrics['execution_accuracy']:.2%} ({metrics['execution_match_count']}/{metrics['total_samples']})")
        print(f"  语义匹配率: {metrics['semantic_match_rate']:.2%} ({metrics['semantic_match_count']}/{metrics['total_samples']})")
        print(f"  平均字符串相似度: {metrics['average_string_similarity']:.2%}")
        
        print(f"\n📈 召回率指标 (Recall / Precision / F1):")
        print(f"  表名召回:")
        print(f"    召回率: {metrics['avg_table_recall']:.2%}")
        print(f"    精确率: {metrics['avg_table_precision']:.2%}")
        print(f"    F1分数: {metrics['avg_table_f1']:.2%}")
        
        print(f"\n  列名召回:")
        print(f"    召回率: {metrics['avg_column_recall']:.2%}")
        print(f"    精确率: {metrics['avg_column_precision']:.2%}")
        print(f"    F1分数: {metrics['avg_column_f1']:.2%}")
        
        print(f"\n  关键词召回:")
        print(f"    召回率: {metrics['avg_keyword_recall']:.2%}")
        print(f"    精确率: {metrics['avg_keyword_precision']:.2%}")
        print(f"    F1分数: {metrics['avg_keyword_f1']:.2%}")
        
        print(f"\n  Token召回:")
        print(f"    召回率: {metrics['avg_token_recall']:.2%}")
        print(f"    精确率: {metrics['avg_token_precision']:.2%}")
        print(f"    F1分数: {metrics['avg_token_f1']:.2%}")
        
        print(f"\n  综合评分:")
        print(f"    召回率: {metrics['avg_overall_recall']:.2%}")
        print(f"    精确率: {metrics['avg_overall_precision']:.2%}")
        print(f"    F1分数: {metrics['avg_overall_f1']:.2%}")
        
        print(f"\n{'='*80}")
        
        # 分析失败案例
        failed_cases = [r for r in self.results if not r["execution_match"]]
        if failed_cases:
            print(f"\n❌ 失败案例 ({len(failed_cases)} 个):")
            for i, case in enumerate(failed_cases[:5], 1):  # 只显示前5个
                print(f"\n  {i}. 问题: {case['question']}")
                print(f"     期望: {case['ground_truth_sql'][:80]}...")
                print(f"     生成: {case['generated_sql'][:80]}...")
                if case['error']:
                    print(f"     错误: {case['error'][:80]}...")
                # 显示召回情况
                rm = case['recall_metrics']
                print(f"     召回: 表{rm['table_recall']:.0%} | 列{rm['column_recall']:.0%} | 关键词{rm['keyword_recall']:.0%}")
    
    def save_results(self, output_file: str):
        """保存评估结果到JSON文件"""
        output_data = {
            "metrics": self._calculate_metrics(),
            "details": self.results
        }
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, ensure_ascii=False, indent=2)
        
        print(f"\n💾 评估结果已保存到: {output_file}")


def main():
    """主函数"""
    
    # 配置
    DB_PATH = "90-文档-Data/tourism.db"
    DATASET_PATH = "90-文档-Data/sakila/q2sql_pairs.json"
    API_KEY = "sk-34b4f***05bb4f"
    OUTPUT_FILE = "05-检索前处理-PreRetrieval/01-查询构建/Text2SQL/evaluation_results.json"
    
    # 检查数据库文件
    if not os.path.exists(DB_PATH):
        print(f"❌ 数据库文件不存在: {DB_PATH}")
        print("请确保 sakila.db 文件存在")
        return
    
    # 加载数据集
    print(f"📂 加载评估数据集: {DATASET_PATH}")
    with open(DATASET_PATH, 'r', encoding='utf-8') as f:
        dataset = json.load(f)
    
    print(f"✓ 加载了 {len(dataset)} 个问答对")
    
    # 创建评估器
    evaluator = Text2SQLEvaluator(DB_PATH, API_KEY)
    
    # 运行评估（可以设置 max_samples 来限制评估数量，加快测试）
    # 完整评估: metrics = evaluator.evaluate_dataset(dataset)
    # 快速测试: metrics = evaluator.evaluate_dataset(dataset, max_samples=5)
    
    metrics = evaluator.evaluate_dataset(dataset, max_samples=10)  # 先测试10个样本
    
    # 保存结果
    evaluator.save_results(OUTPUT_FILE)
    
    print(f"\n✅ 评估完成！")


if __name__ == "__main__":
    main()



📂 加载评估数据集: 90-文档-Data/sakila/q2sql_pairs.json
✓ 加载了 36 个问答对

开始评估 10 个样本

[1/10] 
问题: List all actors with their IDs and names.
标准SQL: SELECT actor_id, first_name, last_name FROM actor;
生成SQL: select actor_id, actor_name from actors

评估结果:
  ✓ 精确匹配: 否
  ✓ 字符串相似度: 79.55%
  ✓ SQL有效性: 否
  ✓ 执行结果匹配: 否

  📊 召回率指标:
    - 表名召回率: 0.00% (F1: 0.00%)
    - 列名召回率: 33.33% (F1: 40.00%)
    - 关键词召回率: 100.00% (F1: 100.00%)
    - Token召回率: 50.00% (F1: 54.55%)
    - 综合召回率: 40.00% (F1: 43.22%)

[2/10] 
问题: Add a new actor named 'John Doe'.
标准SQL: INSERT INTO actor (first_name, last_name) VALUES ('John', 'Doe');
生成SQL: insert into actors (name) values ('john doe')

评估结果:
  ✓ 精确匹配: 否
  ✓ 字符串相似度: 80.73%
  ✓ SQL有效性: 否
  ✓ 执行结果匹配: 否

  📊 召回率指标:
    - 表名召回率: 100.00% (F1: 0.00%)
    - 列名召回率: 100.00% (F1: 100.00%)
    - 关键词召回率: 100.00% (F1: 100.00%)
    - Token召回率: 62.50% (F1: 66.67%)
    - 综合召回率: 92.50% (F1: 75.85%)

[3/10] 
问题: Update the last name of actor with ID 1 to 'Smith'.
标准SQL: UPDATE actor SET last_na